# Caesar Cipher

In [50]:
# Encrypt Function
def caesar_encrypt(text, shift):
    result = ""
    for char in text:
        if char.isalpha():
            base = ord('A') if char.isupper() else ord('a')
            result += chr((ord(char) - base + shift) % 26 + base)
        else:
            result += char
    return result

In [51]:
# Decrypt Function
def caesar_decrypt(text, shift):
    return caesar_encrypt(text, -shift)

In [52]:
# Brute-Force Attack
def caesar_bruteforce(cipher_text):
    for shift in range(26):
        decrypted = caesar_decrypt(cipher_text, shift)
        print(f"Shift {shift:2d}: {decrypted}")

In [53]:
# Example
message = "NCS Assignment 1"
cipher = caesar_encrypt(message, 3)
print(cipher)

plain = caesar_decrypt(cipher, 3)
print(plain)

caesar_bruteforce("Ceaser Cipher")

QFV Dvvljqphqw 1
NCS Assignment 1
Shift  0: Ceaser Cipher
Shift  1: Bdzrdq Bhogdq
Shift  2: Acyqcp Agnfcp
Shift  3: Zbxpbo Zfmebo
Shift  4: Yawoan Yeldan
Shift  5: Xzvnzm Xdkczm
Shift  6: Wyumyl Wcjbyl
Shift  7: Vxtlxk Vbiaxk
Shift  8: Uwskwj Uahzwj
Shift  9: Tvrjvi Tzgyvi
Shift 10: Suqiuh Syfxuh
Shift 11: Rtphtg Rxewtg
Shift 12: Qsogsf Qwdvsf
Shift 13: Prnfre Pvcure
Shift 14: Oqmeqd Oubtqd
Shift 15: Npldpc Ntaspc
Shift 16: Mokcob Mszrob
Shift 17: Lnjbna Lryqna
Shift 18: Kmiamz Kqxpmz
Shift 19: Jlhzly Jpwoly
Shift 20: Ikgykx Iovnkx
Shift 21: Hjfxjw Hnumjw
Shift 22: Giewiv Gmtliv
Shift 23: Fhdvhu Flskhu
Shift 24: Egcugt Ekrjgt
Shift 25: Dfbtfs Djqifs


# Playfair Cipher

In [54]:
# Build the 5×5 Key Matrix
import string

def generate_key_matrix(key):
    key = key.lower().replace("j", "i")
    matrix = []
    used = set()

    for char in key:
        if char.isalpha() and char not in used:
            used.add(char)
            matrix.append(char)

    # Add remaining letters (a–z excluding j)
    for char in string.ascii_lowercase:
        if char != "j" and char not in used:
            used.add(char)
            matrix.append(char)

    # Convert list to 5x5 matrix
    return [matrix[i*5:(i+1)*5] for i in range(5)]

In [55]:
# Locate a character in the matrix
def find_position(matrix, char):
    for r in range(5):
        for c in range(5):
            if matrix[r][c] == char:
                return r, c
    return None

In [56]:
# Prepare plaintext (digraphs, X padding)
def prepare_text(text):
    text = text.lower().replace(" ", "").replace("j", "i")
    prepared = ""
    i = 0

    while i < len(text):
        a = text[i]
        b = text[i+1] if i+1 < len(text) else "x"

        if a == b:
            prepared += a + "x"
            i += 1
        else:
            prepared += a + b
            i += 2

    if len(prepared) % 2 != 0:
        prepared += "x"

    return prepared

In [57]:
# Encrypt digraph using Playfair rules
def playfair_encrypt_pair(matrix, a, b):
    r1, c1 = find_position(matrix, a)
    r2, c2 = find_position(matrix, b)

    # Same row → shift right
    if r1 == r2:
        return matrix[r1][(c1 + 1) % 5] + matrix[r2][(c2 + 1) % 5]

    # Same column → shift down
    if c1 == c2:
        return matrix[(r1 + 1) % 5][c1] + matrix[(r2 + 1) % 5][c2]

    # Rectangle → swap columns
    return matrix[r1][c2] + matrix[r2][c1]

In [58]:
# Decrypt digraph
def playfair_decrypt_pair(matrix, a, b):
    r1, c1 = find_position(matrix, a)
    r2, c2 = find_position(matrix, b)

    # Same row → shift left
    if r1 == r2:
        return matrix[r1][(c1 - 1) % 5] + matrix[r2][(c2 - 1) % 5]

    # Same column → shift up
    if c1 == c2:
        return matrix[(r1 - 1) % 5][c1] + matrix[(r2 - 1) % 5][c2]

    # Rectangle → swap columns
    return matrix[r1][c2] + matrix[r2][c1]

In [59]:
# Full encryption function
def playfair_encrypt(text, key):
    matrix = generate_key_matrix(key)
    prepared = prepare_text(text)
    cipher = ""

    for i in range(0, len(prepared), 2):
        cipher += playfair_encrypt_pair(matrix, prepared[i], prepared[i+1])

    return cipher

In [60]:
# Full decryption function
def playfair_decrypt(cipher, key):
    matrix = generate_key_matrix(key)
    plain = ""

    for i in range(0, len(cipher), 2):
        plain += playfair_decrypt_pair(matrix, cipher[i], cipher[i+1])

    return plain

In [61]:
#Example
key = "COMPUTER"
plaintext = "ENEMYONTARGET"

cipher = playfair_encrypt(plaintext, key)
print("Cipher:", cipher)

decoded = playfair_decrypt(cipher, key)
print("Decrypted:", decoded)

Cipher: rlrowpkrbafrrv
Decrypted: enemyontargetx


# Hill Cipher

In [62]:
import numpy as np

In [63]:
# Convert letter → number and number → letter

def char_to_num(c):
    return ord(c) - ord('A')

def num_to_char(n):
    return chr(n + ord('A'))

In [64]:
# Prepare text (remove spaces, pad as needed)

def prepare_text(text, n):
    text = text.replace(" ", "").upper()
    while len(text) % n != 0:
        text += "X"   # padding
    return text

In [65]:
# Modular inverse of matrix (mod 26)

def mod_inverse_matrix(matrix, modulus=26):
    det = int(np.round(np.linalg.det(matrix)))
    det = det % modulus

    # modular inverse of determinant
    det_inv = None
    for i in range(26):
        if (det * i) % modulus == 1:
            det_inv = i
            break
    if det_inv is None:
        raise ValueError("Key matrix is not invertible modulo 26.")

    # adjugate matrix
    adj = np.round(det * np.linalg.inv(matrix)).astype(int) % modulus

    # inverse matrix modulo 26
    return (det_inv * adj) % modulus

In [66]:
# Encrypt

def hill_encrypt(plaintext, key_matrix):
    n = key_matrix.shape[0]
    plaintext = prepare_text(plaintext, n)
    cipher = ""

    for i in range(0, len(plaintext), n):
        block = plaintext[i:i+n]
        vector = np.array([char_to_num(c) for c in block])
        encrypted = key_matrix.dot(vector) % 26
        cipher += ''.join(num_to_char(int(x)) for x in encrypted)

    return cipher

In [67]:
# Decrypt

def hill_decrypt(ciphertext, key_matrix):
    n = key_matrix.shape[0]
    inv_key = mod_inverse_matrix(key_matrix)
    plain = ""

    for i in range(0, len(ciphertext), n):
        block = ciphertext[i:i+n]
        vector = np.array([char_to_num(c) for c in block])
        decrypted = inv_key.dot(vector) % 26
        plain += ''.join(num_to_char(int(x)) for x in decrypted)

    return plain

In [68]:
# Example 3*3 key matrix

import numpy as np

# Example key matrix (must be invertible mod 26)
K = np.array([[6, 24, 1],
              [13, 16, 10],
              [20, 17, 15]])

plaintext = "HELLO WORLD"

cipher = hill_encrypt(plaintext, K)
print("Cipher:", cipher)

decoded = hill_decrypt(cipher, K)
print("Decrypted:", decoded)

Cipher: TFJIPIJSGVNQ
Decrypted: GHNTQLUYIHWT
